In [1]:
require './GenericConvolution.rb'
require './WaveletFilters.rb'

true

In [2]:
openmp = false

def Kinetic1d(filter, optims=GenericOptimization::new)
  conv_operation = GenericConvolutionOperator1d::new(filter, :kinetic => :inplace, :ld => true, 
    :narr => true, :a_x => true,:a_y=>true, :a => true, :dot_in => true)
  conv_operation.optimize(optims) if optims

  p, subops = conv_operation.procedure

  kernel = BOAST::CKernel::new

  print_header

  subops.each_value { |op|
    BOAST::pr op
    puts "chosen:"+ op.name
  }
  BOAST::pr p

  kernel.procedure = p
  kernel.cost_function = lambda { |*args| conv_operation.cost(*args) }
  return kernel
end


:Kinetic1d

In [3]:
n1 = 32
n2 = 32
n3 = 32

optims = GenericOptimization::new(:repeat => 1,
  :unroll_range => [1,1], :dimensions => [n1,n2,n3],:openmp => openmp, :mod_arr_test => false,
  :tt_arr_test => false)
conv_filter = ConvolutionFilter::new('sym8_d2',SYM8_D2,14)
5 #to output

5

In [5]:
kkin8 = Kinetic1d( conv_filter, optims )
5

d_sym8_d2_p_10_dotin_ld_u1_v1_1_t_f_t: 0.6956960000000001 ms 2.7318598928267517 GFlops
d_sym8_d2_p_10_a_dotin_ld_u1_v1_1_t_f_t: 0.722332 ms 2.6311225309137627 GFlops
d_sym8_d2_p_10_ain_dotin_ld_u1_v1_1_t_f_t: 0.7228220000000001 ms 2.6293388967131603 GFlops
d_sym8_d2_p_10_ay_dotin_ld_u1_v1_1_t_f_t: 0.6928770000000001 ms 2.742974582790308 GFlops
d_sym8_d2_p_10_a_ain_dotin_ld_u1_v1_1_t_f_t: 0.739224 ms 2.5709987770959817 GFlops
d_sym8_d2_p_10_a_ay_dotin_ld_u1_v1_1_t_f_t: 0.7495740000000001 ms 2.535498830002108 GFlops
d_sym8_d2_p_10_ain_ay_dotin_ld_u1_v1_1_t_f_t: 0.73482 ms 2.5864075555918453 GFlops
d_sym8_d2_p_10_a_ain_ay_dotin_ld_u1_v1_1_t_f_t: 0.736094 ms 2.581931112059057 GFlops
d_sym8_d2_p_01_dotin_ld_u1_v1_0_t_f_t: 0.975513 ms 1.9482508177748528 GFlops
d_sym8_d2_p_01_a_dotin_ld_u1_v1_0_t_f_t: 0.9690920000000001 ms 1.961159518394538 GFlops
d_sym8_d2_p_01_ain_dotin_ld_u1_v1_0_t_f_t: 1.0016880000000001 ms 1.8973412879060145 GFlops
d_sym8_d2_p_01_ay_dotin_ld_u1_v1_0_t_f_t: 0.953307 ms 1.

5

In [6]:
#puts kkin8
kkin8.build(:openmp => openmp)
5

5

In [7]:
require './Kinetic.rb'

true

In [8]:
kref = kinetic_per_ref_optim
5

5

In [9]:
kref.build(:openmp => openmp)
5

5

In [10]:
input = ANArray.float(32,n1,n2,n3).random! #fill!(1.0)
output_ref = ANArray.float(32,n1,n2,n3)
output = ANArray.float(32,n1,n2,n3)
hgrid = NArray.float(3)
hgrid[0] = 0.5
hgrid[1] = 0.6
hgrid[2] = 0.7
c=0.5

0.5

In [11]:
stats = kref.run(n1-1, n2-1, n3-1, hgrid, input, output_ref, c)

{:duration=>0.001260262, :energy=>{:"0.package-0"=>0.037415, :"0.core"=>0.035766, :"0.uncore"=>0.0, :"0.dram"=>0.0035399999999999997}}

In [12]:
puts input[0]
output_ref[0]

0.9026809755342587


6.066264643565508

In [13]:
ndims=NArray.int(3)
ndims[0]=n1
ndims[1]=n2
ndims[2]=n3
bc = BC::PERIODIC
c=0.5
eval=0.0
scal=NArray.float(3)
(0..2).each{ |ind| scal[ind] = -0.5 / (hgrid[ind]*hgrid[ind])   }

0..2

In [1]:
#kkin8

1D operation in the x direction

In [14]:
stat=kkin8.run(3,0,ndims,bc,ndims,ndims,1,input,output,scal[0],c,0.0,eval) # out= scal[0] * d2x [in] + c * in + 0.0 * out

{:reference_return=>{:dot_in=>19729.2606953027}, :duration=>0.0007462730000000001, :energy=>{:"0.package-0"=>0.0, :"0.core"=>0.0, :"0.uncore"=>0.0, :"0.dram"=>0.0}}

In [15]:
eval=stat[:reference_return][:dot_in]
puts eval

19729.2606953027


y and z direction

In [16]:
kkin8.run(3,1,ndims,bc,ndims,ndims,1,input,output,scal[1],0.0,1.0,eval) # out= scal[1]* d2y[in]  + 1.0 * out
eval+=stat[:reference_return][:dot_in]
kkin8.run(3,2,ndims,bc,ndims,ndims,1,input,output,scal[2],0.0,1.0,eval) # out= scal[2]* d2z[in]  + 1.0 * out
eval+=stat[:reference_return][:dot_in]


59187.7820859081

In [17]:
puts input[0]
output[0]

0.9026809755342587


6.066264643565508

In [18]:
eval*hgrid[0]*hgrid[1]*hgrid[2]

12429.4342380407

In [19]:
epsilon=1.0e-13
diff = (output_ref - output).abs
diff.each { |elem|
  raise "Warning: residue too big: #{elem}" if elem > epsilon
}
puts 'ok'

ok


In [36]:
File.open('Kinetic8.f90','w'){ | f |
  f.write(kkin8.to_s)
  }

915430